In [71]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle
import copy

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss

from sklearn.metrics import r2_score

import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [211]:
path = '../DATA'
trainFile = 'train_augmented.csv'
testFile = 'test_augmented.csv'

In [280]:
# Load all data
train_df = pd.read_csv(os.path.join(path, trainFile),
                       header = 0,
                       index_col='id')
test_df = pd.read_csv(os.path.join(path, testFile),
                      header = 0,
                      index_col='id')


## Transform categorical (and ordinal) variables

In [259]:
# occupation, marital status, state, region, race, urban are categorical variables, education is an ordinal variable.
train_df['educationCat'] = train_df['education']
test_df['educationCat'] = test_df['education']


catCols = ['region', 'state', 'marital', 'occupation','urban', 'race']

#xgboost expects categoricals starting at 0
train_df[catCols] = train_df[catCols] - 1
test_df[catCols] = test_df[catCols] - 1

catCols.append('educationCat')
train_df[catCols] = train_df[catCols].astype('category')
test_df[catCols] = test_df[catCols].astype('category')

## Set data types to float

In [260]:
numericalCols = ['year', 'month', 'age', 'education', 'familysize', 'income','daysInMonth',
                     'daysInPrevMonth', 'CPIUrban', 'LaborForceParticip',
                     'UnemploymentRateByAge', 'AnnualExpenditureByAge',
                     'AnnualExpenditureByIncome', 'AnnualExpenditureByFamilySize']

# Convert to float
train_df[numericalCols] = train_df[numericalCols].astype('float')
test_df[numericalCols] = test_df[numericalCols].astype('float')

## Feature Engineering

Due to time constraints, I will skip feature selection. More on this with xgb!

In [ ]:
# features added from earlier version of code, based on vanilla sklearn quantile regression

train_df['education_per_familySize'] = train_df['education']/train_df['familysize']
train_df['age_per_familySize'] =  train_df['age']/train_df['familysize']
train_df['education_per_income'] =  train_df['education']/(train_df['income']+1)

test_df['education_per_familySize'] = test_df['education']/test_df['familysize']
test_df['age_per_familySize'] =  test_df['age']/test_df['familysize']
test_df['education_per_income'] =  test_df['education']/(test_df['income']+1)

numericalCols.extend(['education_per_familySize',
                'age_per_familySize',
                'education_per_income'])

# Convert to float
train_df[numericalCols] = train_df[numericalCols].astype('float')
test_df[numericalCols] = test_df[numericalCols].astype('float')

### Get individual feature mutual information

In [62]:
from sklearn.feature_selection import mutual_info_regression as mi_reg

catCols_nonNull = ['marital', 'urban', 'race', 'educationCat']
mi_cat = mi_reg(X_train[catCols_nonNull], y_train, discrete_features=True)

mi_cat_nulls = []
for cols in ['region', 'state', 'occupation']:
    X_tmp = X_train[cols]
    y_tmp = y_train[X_tmp.notna()]
    X_tmp = X_tmp[X_tmp.notna()]
    mi_cat_nulls.append(mi_reg(X_tmp.to_frame(), y_tmp, discrete_features=True))
mi_cat_nulls = np.array(mi_cat_nulls).flatten()

numericalCols = ['year', 'month', 'age', 'education', 'familysize', 'income','daysInMonth',
                     'daysInPrevMonth', 'CPIUrban', 'LaborForceParticip',
                     'UnemploymentRateByAge', 'AnnualExpenditureByAge',
                     'AnnualExpenditureByIncome', 'AnnualExpenditureByFamilySize']
numericalCols.extend(['education_per_familySize','age_per_familySize','education_per_income'])
mi_num = mi_reg(X_train[numericalCols], y_train, discrete_features=False)

In [81]:
cols_all = copy.copy(catCols_nonNull)
cols_all.extend(['region', 'state', 'occupation'])
cols_all.extend(numericalCols)
mis = np.concatenate((mi_cat, mi_cat_nulls, mi_num), axis=0)
mi_df = pd.DataFrame(data={'feature':cols_all, 'mi': mis})

mi_df.head(len(cols_all))

,feature,mi
0,marital,0.084945
1,urban,0.003023
2,race,0.011005
3,educationCat,0.074966
4,region,0.006312
5,state,0.020226
6,occupation,0.056487
7,year,0.021980
8,month,0.011270
9,age,0.060308


### Get crosses (multiplication, division) for numerical features and their mutual information

I evaluated how helpful a feature may be via a mutual information-based heuristic (to save time compared to using the model to select cross features).


In [ ]:
c1 = []
c1_val = []
c2_val = []
c2 = []
p = []
d1 = []
d2 = []
for i1, col1 in enumerate(numericalCols):
    for i2, col2 in enumerate(numericalCols):
        if i2 < i1:
            continue
        prod = (X_train[col1]+1)*(X_train[col2]+1)
        div1 = (X_train[col1]+1)/(X_train[col2]+1)
        div2 = (X_train[col2]+1)/(X_train[col1]+1)
        c1.append(col1)
        c1_val.append(mi_df.loc[mi_df['feature']==col1,:]['mi'].values[0])
        c2_val.append(mi_df.loc[mi_df['feature']==col2,:]['mi'].values[0])
        c2.append(col2)
        p.append(mi_reg(prod.to_frame(), y_train, discrete_features=False))
        d1.append(mi_reg(prod.to_frame(), y_train, discrete_features=False))
        d2.append(mi_reg(prod.to_frame(), y_train, discrete_features=False))
        

In [92]:
mi_df_num_cross = pd.DataFrame(data={'c1':c1, 'c2':c2, 'c1_val':c1_val, 'c2_val':c2_val, 'p':p, 'd1':d1, 'd2': d2})

In [112]:
mi_df_num_cross['improvement_p'] = mi_df_num_cross['p'] - mi_df_num_cross[['c1_val', 'c2_val']].max(axis=1)
mi_df_num_cross['improvement_d1'] = mi_df_num_cross['d1'] - mi_df_num_cross[['c1_val', 'c2_val']].max(axis=1)
mi_df_num_cross['improvement_d2'] = mi_df_num_cross['d2'] - mi_df_num_cross[['c1_val', 'c2_val']].max(axis=1)
mi_df_num_cross = mi_df_num_cross.sort_values(by=['improvement_d2'], ascending=False)
with pd.option_context('display.max_rows', None):  
    display(mi_df_num_cross)

,c1,c2,c1_val,c2_val,p,d1,d2,improvement_p,improvement_d1,improvement_d2
58,education,AnnualExpenditureByFamilySize,0.079910,0.111136,[0.20130854532278075],[0.2015601665459128],[0.20189391211839158],[0.0901729988076978],[0.09042462003082985],[0.09075836560330863]
49,education,familysize,0.079910,0.082344,[0.1644119544841942],[0.16553796247686137],[0.16530373805327603],[0.08206829249524406],[0.08319430048791121],[0.08296007606432587]
69,familysize,AnnualExpenditureByAge,0.082344,0.076354,[0.14311665730090883],[0.14295186092157763],[0.14308367558536528],[0.06077299531195868],[0.06060819893262748],[0.06074001359641512]
56,education,AnnualExpenditureByAge,0.079910,0.076354,[0.13901528056569923],[0.1389896835860851],[0.138386525600672],[0.059105028170156615],[0.059079431190542486],[0.05847627320512938]
66,familysize,CPIUrban,0.082344,0.043453,[0.12573229115339046],[0.1260110348617811],[0.12607779296106614],[0.04338862916444031],[0.043667372872830956],[0.04373413097211598]
61,education,education_per_income,0.079910,0.270080,[0.30934058261366637],[0.30964088537137435],[0.3092659866208738],[0.03926019170452477],[0.039560494462232754],[0.03918559571173219]
10,year,UnemploymentRateByAge,0.021980,0.024438,[0.06214664074717202],[0.062134347337459594],[0.06216914604247048],[0.03770892163428741],[0.03769662822457498],[0.037731426929585865]
118,LaborForceParticip,UnemploymentRateByAge,0.023345,0.024438,[0.062394282963177616],[0.0620216390807693],[0.06171847163404642],[0.037956563850293],[0.03758391996788468],[0.03728075252116181]
4,year,familysize,0.021980,0.082344,[0.1112212389425018],[0.11175625046750692],[0.1109760114867333],[0.028877576953551642],[0.029412588478556767],[0.02863234949778315]
67,familysize,LaborForceParticip,0.082344,0.023345,[0.11014444129576706],[0.10931825041263554],[0.11010836768887167],[0.027800779306816903],[0.026974588423685386],[0.02776470569992151]


Will use the first few with a function (multiplication or division) that seems meaningful.
Specifically: 
* AnnualExpenditureByFamilySize per education
* AnnualExpenditureByAge per familysize
* CPIUrban (inflation) times familySize

As noted above, a previous version of the code tested other pre-augmentation crosses on vanilla quantile regression and found these to be relevant:
* education per familySize
* age per familySize
* education per income


### Get crosses (conjunction) for categorical features and their mutual information

In [ ]:
c1 = []
c1_val = []
c2_val = []
c2 = []
p = []
for i1, col1 in enumerate(catCols):
    for i2, col2 in enumerate(catCols):
        if i2 <= i1:
            continue
            v1 = X_train[col1].dropna().unique()
            v2 = X_train[col2].dropna().unique()
            v1 = pd.Series(v1, name = col1)
            v2 = pd.Series(v2, name = col2)
            c_df = pd.merge(v1, v2,how='cross')
            c_df['index1'] = c_df.index
            y_train2 = y_train[(X_train[col1].notna()) & (X_train[col2].notna())]
            X_train2 = X_train[(X_train[col1].notna()) & (X_train[col2].notna())]
            m_df = pd.merge(X_train2, c_df, on=[col1, col2])
            c1.append(col1)
            c1_val.append(mi_df.loc[mi_df['feature']==col1,:]['mi'].values[0])
            c2_val.append(mi_df.loc[mi_df['feature']==col2,:]['mi'].values[0])
            c2.append(col2)
            p.append(mi_reg(m_df['index1'].to_frame(), y_train2, discrete_features=True))
            

In [132]:
mi_df_cat_cross = pd.DataFrame(data={'c1':c1, 'c2':c2, 'c1_val':c1_val, 'c2_val':c2_val, 'p':p})
mi_df_cat_cross['improvement_p'] = mi_df_cat_cross['p'] - mi_df_cat_cross[['c1_val', 'c2_val']].max(axis=1)
mi_df_cat_cross = mi_df_cat_cross.sort_values(by=['improvement_p'], ascending=False)
with pd.option_context('display.max_rows', None):  
    display(mi_df_cat_cross)

,c1,c2,c1_val,c2_val,p,improvement_p
3,region,urban,0.006312,0.003023,[0],[-0.006312160774230247]
18,urban,race,0.003023,0.011005,[0.0013720779019432694],[-0.009632990355563864]
4,region,race,0.006312,0.011005,[0],[-0.011005068257507133]
0,region,state,0.006312,0.020226,[0],[-0.020226115694808477]
8,state,urban,0.020226,0.003023,[0],[-0.020226115694808477]
9,state,race,0.020226,0.011005,[0],[-0.020226115694808477]
2,region,occupation,0.006312,0.056487,[0.0021239435450493005],[-0.05436303760781902]
7,state,occupation,0.020226,0.056487,[0.0012639514716399347],[-0.055223029681228386]
15,occupation,urban,0.056487,0.003023,[0.0011314227205860838],[-0.05535555843228224]
16,occupation,race,0.056487,0.011005,[0.0009707974564427957],[-0.055516183696425525]


None of these seem like they would help.

### Add in new features

Note that we will also be:
* applying a cyclical transformation to month
* replacing high cardinality categories with target encoding

In [319]:
# Start over
# Reload all data
train_df = pd.read_csv(os.path.join(path, trainFile),
                       header = 0,
                       index_col='id')
test_df = pd.read_csv(os.path.join(path, testFile),
                      header = 0,
                      index_col='id')

# occupation, marital status, state, region, race, urban are categorical variables, education is an ordinal variable.
train_df['educationCat'] = train_df['education']
test_df['educationCat'] = test_df['education']


catCols = ['region', 'state', 'marital', 'occupation','urban', 'race']

#xgboost expects categoricals starting at 0
train_df[catCols] = train_df[catCols] - 1
test_df[catCols] = test_df[catCols] - 1

catCols.append('educationCat')
train_df[catCols] = train_df[catCols].astype('category')
test_df[catCols] = test_df[catCols].astype('category')

numericalCols = ['year', 'month', 'age', 'education', 'familysize', 'income','daysInMonth',
                     'daysInPrevMonth', 'CPIUrban', 'LaborForceParticip',
                     'UnemploymentRateByAge', 'AnnualExpenditureByAge',
                     'AnnualExpenditureByIncome', 'AnnualExpenditureByFamilySize']

# Convert to float
train_df[numericalCols] = train_df[numericalCols].astype('float')
test_df[numericalCols] = test_df[numericalCols].astype('float')

In [320]:
numericalCols = ['year', 'month', 'age', 'education', 'familysize', 'income','daysInMonth',
                     'daysInPrevMonth', 'CPIUrban', 'LaborForceParticip',
                     'UnemploymentRateByAge', 'AnnualExpenditureByAge',
                     'AnnualExpenditureByIncome', 'AnnualExpenditureByFamilySize']

catCols = ['region', 'state', 'marital', 'occupation','urban', 'race','educationCat']

newNumCols = []
remNumCols = []
remCatCols = []
# circular encoding of months
train_df['month_s'] = np.sin(2 * np.pi * (train_df['month']-1)/11.0)
train_df['month_c'] = np.cos(2 * np.pi * (train_df['month']-1)/11.0)
train_df = train_df.drop(columns=['month'])

test_df['month_s'] = np.sin(2 * np.pi * (test_df['month']-1)/11.0)
test_df['month_c'] = np.cos(2 * np.pi * (test_df['month']-1)/11.0)
test_df = test_df.drop(columns=['month'])

newNumCols.extend(['month_s','month_c'])
remNumCols.append('month')

# target encoding of categorical variables with high cardinality i.e. state and cardinality
f_list = ['mean',
         lambda x: np.quantile(x, 0.1),
         lambda x: np.quantile(x, 0.25),
         lambda x: np.quantile(x, 0.5),
         lambda x: np.quantile(x, 0.75),
         lambda x: np.quantile(x, 0.9)]
# only use train split for aggregation, no leakage
randomState = 0
y = train_df['expense']
X = train_df
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                     y,                                                                                  
                                                     test_size=0.3, 
                                                     random_state=randomState)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, 
                                                        y_train,                                                                                     
                                                        test_size=0.15, 
                                                        random_state=randomState)

for col in ['state','occupation']:
    stats = X_train['expense'].groupby(X_train[col]).agg(f_list)
    train_df = pd.merge(train_df.reset_index(names='id'), stats, how='left', on=[col]).set_index('id')
    train_df = train_df.drop(columns=[col])
    train_df = train_df.rename(columns={'mean': col+'_mean',
                                       '<lambda_0>': col+'_q1',
                                       '<lambda_1>': col+'_q2',
                                       '<lambda_2>': col+'_q3',
                                       '<lambda_3>': col+'_q4',
                                       '<lambda_4>': col+'_q5'})


    test_df = pd.merge(test_df.reset_index(names='id'), stats, how='left', on=[col]).set_index('id')
    test_df = test_df.drop(columns=[col])
    test_df = test_df.rename(columns={'mean': col+'_mean',
                                       '<lambda_0>': col+'_q1',
                                       '<lambda_1>': col+'_q2',
                                       '<lambda_2>': col+'_q3',
                                       '<lambda_3>': col+'_q4',
                                       '<lambda_4>': col+'_q5'})

    newNumCols.extend([col+'_mean',col+'_q1',col+'_q2',col+'_q3',col+'_q4',col+'_q5'])    
    remCatCols.append(col)
    
train_df['education_per_familySize'] = train_df['education']/train_df['familysize']
train_df['age_per_familySize'] =  train_df['age']/train_df['familysize']
train_df['education_per_income'] =  train_df['education']/(train_df['income']+1)
train_df['AnnualExpenditureByFamilySize_per_education'] =  train_df['AnnualExpenditureByFamilySize']/(train_df['education']+1)
train_df['AnnualExpenditureByAge_per_familysize'] =  train_df['AnnualExpenditureByAge']/(train_df['familysize'])
train_df['CPIUrban_times_familySize'] = train_df['CPIUrban']*train_df['familysize']

test_df['education_per_familySize'] = test_df['education']/test_df['familysize']
test_df['age_per_familySize'] =  test_df['age']/test_df['familysize']
test_df['education_per_income'] =  test_df['education']/(test_df['income']+1)
test_df['AnnualExpenditureByFamilySize_per_education'] =  test_df['AnnualExpenditureByFamilySize']/(test_df['education']+1)
test_df['AnnualExpenditureByAge_per_familysize'] =  test_df['AnnualExpenditureByAge']/(test_df['familysize'])
test_df['CPIUrban_times_familySize'] = test_df['CPIUrban']*test_df['familysize']

newNumCols.extend(['education_per_familySize',
                'age_per_familySize',
                'education_per_income',
               'AnnualExpenditureByFamilySize_per_education',
               'AnnualExpenditureByAge_per_familysize',
               'CPIUrban_times_familySize'])

train_df = train_df.drop(columns=['daysInMonth'])
train_df = train_df.drop(columns=['daysInPrevMonth'])
test_df = test_df.drop(columns=['daysInMonth'])
test_df = test_df.drop(columns=['daysInPrevMonth'])

remNumCols.extend(['daysInMonth', 'daysInPrevMonth'])

In [321]:
# make sure everything is float
numericalCols.extend(newNumCols)
numericalCols = np.setdiff1d(numericalCols, remNumCols)

train_df[numericalCols] = train_df[numericalCols].astype('float')
test_df[numericalCols] = test_df[numericalCols].astype('float')

catCols = np.setdiff1d(catCols, remCatCols)

In [304]:
train_df.to_csv(path_or_buf=os.path.join(path, 'train_augmented_engineered.csv'), index=True, index_label='id')
test_df.to_csv(path_or_buf=os.path.join(path, 'test_augmented_engineered.csv'), index=True, index_label='id')